# Motor Trend Car Road Test Regression Problem
The data was extracted from the 1974 Motor Trend US magazine and comprises fuel consumption and 10 aspects of automobile design and performance for 32 automobiles (1973--74 models).

We form a regression problem by predicting the MPG of a car from its other features.

*This notebooks compares regression performance for different models on a Leave-One-Out Cross-Validation of the motor trend car test data.*

## Import Dependencies

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

## Load the Motor Trend Car Test Data

In [2]:
df = pd.read_csv('mtcars.csv', header=0)
df = df.drop(columns=["model"]) # Not used in regression
X = np.array(df.iloc[:,1:].values, dtype=float)
y = np.array(df.iloc[:,0].values, dtype=float)

## Normalize the Data
We normalize across the full feature matrix since both X_train and X_test would be available when we train our model.

For the target values, we center when we compute the leave-one-out split and only use the training data to treat y_test as unknown.

Note: The PCA transformation has no effect on least-squares or ridge regression models with a single regularizer, but it is significant when using multiple regularizers.

In [3]:
X = StandardScaler().fit_transform(X)
X = PCA().fit_transform(X)

## Load pre-fitted ridge regresson parameters
A second order optimizer was used to fit ridge regression to the generated regression problems so as to minize the LOOCV and GCV for 1 and p regularization parameters on each leave-one-out training matrix.

In [4]:
loocv_1_params = np.loadtxt('mtcars-loocv-1-params.txt')
loocv_p_params = np.loadtxt('mtcars-loocv-p-params.txt')
gcv_1_params = np.loadtxt('mtcars-gcv-1-params.txt')
gcv_p_params = np.loadtxt('mtcars-gcv-p-params.txt')

## Compute a Leave-One-Out Split

In [5]:
def compute_leave_one_out_split(X, y, index):
    num_data = len(y)
    train_indexes = list(range(index)) + list(range(index+1, num_data))
    test_indexes = [index]
    X_train, y_train = X[train_indexes, :], y[train_indexes]
    X_test, y_test = X[test_indexes, :], y[test_indexes]
    y_mean = np.mean(y_train)
    y_train -= y_mean
    y_test -= y_mean
    return X_train, y_train, X_test, y_test

## Verify Loaded Ridge Regression Parameters Represent for CV Optimums
For each leave-one-out training dataset, we verify that the loaded ridge regression parameters supply a minimum of the LOOCV error by walking to neighboring parameters and confirming that we don't find one that's significantly better.

In [6]:
def compute_loocv(X, y, D):
    A = np.dot(np.conj(X.T), X) + D
    A_inv = np.linalg.inv(A)
    b_hat = np.dot(A_inv, np.dot(np.conj(X.T), y))
    y_hat = np.dot(X, b_hat)
    h = np.array([np.dot(x_i.conj(), np.dot(A_inv, x_i)).real for x_i in X])
    return np.sum((np.abs(y - y_hat) / (1 - h))**2) / len(y)

def verify_loocv_opt_1(X, y, alpha):
    delta_x = 1.0e-3
    def f(x):
        return compute_loocv(X, y, np.identity(X.shape[1])*x)
    f0 = f(alpha)
    for x in [alpha-delta_x, alpha+delta_x]:
        x = np.abs(x)
        delta_y = f(x) - f0
        relative_delta_y = delta_y / delta_x
        if relative_delta_y < 0 and np.abs(relative_delta_y) > 1.0e-3:
            assert False, "can't verify optimum"

def verify_loocv_opt_p(X, y, alpha):
    delta_x = 1.0e-3
    for i, alpha_i in enumerate(alpha):
        def f(x):
            alpha_copy = np.array(alpha)
            alpha_copy[i] = x
            return compute_loocv(X, y, np.diag(alpha_copy))
        f0 = f(alpha_i)
        for x in [alpha_i - delta_x, alpha_i + delta_x]:
            x = np.abs(x)
            delta_y = f(x) - f0
            relative_delta_y = delta_y / delta_x
            if relative_delta_y < 0 and np.abs(relative_delta_y) > 1.0e-3:
                assert False, "can't verify optimum"

for index in range(len(y)):
    X_train, y_train, _, _ = compute_leave_one_out_split(X, y, index)
    verify_loocv_opt_1(X_train, y_train, loocv_1_params[index])
    verify_loocv_opt_p(X_train, y_train, loocv_p_params[index])
    U, S, Vt = np.linalg.svd(X_train)
    n = len(y_train)
    W = [[np.exp(2j*np.pi*i*j / n) / np.sqrt(n) for j in range(n)] for i in range(n)]
    Q = np.dot(W, U.T)
    X_train_prime = np.dot(Q, X_train)
    y_train_prime = np.dot(Q, y_train)
    verify_loocv_opt_1(X_train_prime, y_train_prime, gcv_1_params[index])
    verify_loocv_opt_p(X_train_prime, y_train_prime, gcv_p_params[index])

## Fit Ridge Regression

In [7]:
def fit_ridge_regression(X, y, alpha):
    if isinstance(alpha, float):
        alpha = np.ones(X.shape[1])*alpha
    A = np.dot(X.T, X) + np.diag(alpha)
    A_inv = np.linalg.inv(A)
    return np.dot(A_inv, np.dot(X.T, y))

## Compute LOOCV Errors

In [8]:
def fit_predict(model_name, index, X_train, y_train, X_test):
    if model_name == "LS":
        # Note: y was already centered during the leave-one-out split
        model = LinearRegression(fit_intercept=False)
        model.fit(X_train, y_train)
        return model.predict(X_test)
    if model_name == "RR-LOOCV-1":
        alpha = loocv_1_params[index]
    elif model_name == "RR-LOOCV-p":
        alpha = loocv_p_params[index]
    elif model_name == "RR-GCV-1":
        alpha = gcv_1_params[index]
    elif model_name == "RR-GCV-p":
        alpha = gcv_p_params[index]
    else:
        assert False, "No such model"
    b_hat = fit_ridge_regression(X_train, y_train, alpha)
    return np.dot(X_test, b_hat)
loocv_errors = defaultdict(list)
model_names = ["LS", "RR-LOOCV-1", "RR-LOOCV-p", "RR-GCV-1", "RR-GCV-p"]
for index in range(len(y)):
    X_train, y_train, X_test, y_test = compute_leave_one_out_split(X, y, index)
    for model_name in model_names:
        y_pred = fit_predict(model_name, index, X_train, y_train, X_test)
        err = y_test - y_pred
        loocv_errors[model_name].append(np.dot(err, err))

## Print the RMSEs from the LOOCV

In [9]:
for model_name in model_names:
    print(model_name.ljust(10), np.sqrt(np.mean(loocv_errors[model_name])))

LS         3.3806726635155115
RR-LOOCV-1 2.7683583580082036
RR-LOOCV-p 2.7735425552077224
RR-GCV-1   2.7494763182212663
RR-GCV-p   2.617569927618669
